In [29]:
#--------------------------------
## IMPORTS
#--------------------------------
import sys
import os

# set the path to find the modules
os.chdir("../005_src")

from config import *

In [30]:
print (os.getenv('CONDA_PREFIX'))

/usr/prakt/s0014/anaconda3/envs/p37_GCN


### Read xml to txt

In [35]:
# TODO: move to functions_xml, DONE 
def getDataRecursive(element):
    data = list()

    # get attributes of element, necessary for all elements
    for key in element.attrib.keys():
        data.append(element.tag + '.' + key + ' = ' + element.attrib.get(key))

    # only end-of-line elements have important text, at least in this example
    if len(element) == 0:
        if element.text is not None:
            data.append(element.tag + ' = ' + element.text)

    # otherwise, go deeper and add to the current tag
    else:
        for el in element:
            within = getDataRecursive(el)

            for data_point in within:
                data.append(element.tag + '.' + data_point)

    return data

In [39]:
def get_flow_details(xml_data, spacing_factor = 5):
    etree = ET.parse(xml_data)
    root = etree.getroot()
    all_rows = getDataRecursive(root)
    spacing = " "*spacing_factor
    return[ f"{spacing} {x}" for x in all_rows if "flow" in x]
    

In [38]:
# read xml file
xml_data = "../004_data/cross_003/cross.rou.xml"
etree = ET.parse(xml_data)
root = etree.getroot()
all_rows = getDataRecursive(root)
spacing = " "*5
flow_rows = [ f"{spacing} {x}" for x in all_rows if "flow" in x]
flow_rows

['      routes.flow.id = flow1',
 '      routes.flow.begin = 0',
 '      routes.flow.end = 2000',
 '      routes.flow.number = 300',
 '      routes.flow.route = routedist2']

In [40]:
get_flow_details(xml_data, spacing_factor = 5)

['      routes.flow.id = flow1',
 '      routes.flow.begin = 0',
 '      routes.flow.end = 2000',
 '      routes.flow.number = 300',
 '      routes.flow.route = routedist2']

### Read txt file to dictionary

In [9]:
txt_data = "../004_data/dataframes/df_sim_000005.txt"
d = {}
with open(f"{txt_data}") as f:
    for line in f:
        if line != "\n":
            key, val,*other = line.split(": ")
            d[key] = (val.strip())

In [10]:
d

{'max_num_veh': '10',
 'date': '20210704-',
 'time': '12h04m16s',
 'usr': '/usr/prakt/s0014',
 'env': '/usr/prakt/s0014/anaconda3/envs/carla_sumo_env',
 'sim_duration_timesteps': '200',
 'sim_duration_actualseconds': '0.4814896583557129',
 'savestat': 'True',
 'plotstat': 'True',
 'conf_ID': '003',
 'conf_file': '../004_data/cross_003/cross.rou.xml',
 '      routes.flow.id': 'flow1',
 '      routes.flow.begin': '0',
 '      routes.flow.end': '2000',
 '      routes.flow.number': '300',
 '      routes.flow.route': 'routedist2',
 'path_figure': '../004_data/figures/plots_data/000005_veh_per_timestep.png',
 'path_df': '../004_data/dataframes/df_sim_000005.zip'}

In [15]:
def read_txt_data(txt_data, split_char = ": "):
    """
    read files such as "../004_data/dataframes/df_sim_000005.txt"
    into a dictionary d['max_num_veh'] = '10'
    """
    d = {}
    with open(f"{txt_data}") as f:
        for line in f:
            if line != "\n":
                key, val,*other = line.split(split_char)
                d[key] = (val.strip())
    return d 

In [14]:
d = read_sim_data(txt_data)
int(d['max_num_veh'])

10

In [10]:
txt_data = "../004_data/GCN_input/20210704-13h44m33s_timesteps200_ec3500_em7000.txt"
d = read_txt_data(txt_data)
d

{'max_num_veh': '10',
 'date': '20210704-',
 'time': '13h44m33s',
 'usr': '/usr/prakt/s0014',
 'env': '/usr/prakt/s0014/anaconda3/envs/p37_GCN',
 'path_input_df': '/storage/remote/atcremers50/ss21_multiagentcontrol/004_data/dataframes/df_sim_000005.zip',
 'info_input_df': '/storage/remote/atcremers50/ss21_multiagentcontrol/004_data/dataframes/df_sim_000005.txt',
 'sim_duration_timesteps': '200',
 'edge_creation_radius': '3500',
 'edge_maintenance_radius': '7000',
 'edge_weights_option': '2',
 'predict_after_timesteps': '2',
 'savestat': 'True',
 'delete_temporary_files': 'True',
 'plotstat': 'True',
 'path_GCN_input': "<_io.BufferedWriter name='../004_data/GCN_input/20210704-13h44m33s_timesteps200_ec3500_em7000.pkl'>",
 'path_GIF': '../004_data/gifs/20210704-13h44m33s_em7000_ec3500_opt_2_graph.gif'}

In [18]:
read_txt_data(d['info_input_df'])

{'max_num_veh': '10',
 'date': '20210704-',
 'time': '12h04m16s',
 'usr': '/usr/prakt/s0014',
 'env': '/usr/prakt/s0014/anaconda3/envs/carla_sumo_env',
 'sim_duration_timesteps': '200',
 'sim_duration_actualseconds': '0.4814896583557129',
 'savestat': 'True',
 'plotstat': 'True',
 'conf_ID': '003',
 'conf_file': '../004_data/cross_003/cross.rou.xml',
 '      routes.flow.id': 'flow1',
 '      routes.flow.begin': '0',
 '      routes.flow.end': '2000',
 '      routes.flow.number': '300',
 '      routes.flow.route': 'routedist2',
 'path_figure': '../004_data/figures/plots_data/000005_veh_per_timestep.png',
 'path_df': '../004_data/dataframes/df_sim_000005.zip'}

In [4]:
GCN_INPUT_FOLDER

'/storage/remote/atcremers50/ss21_multiagentcontrol/004_data/GCN_input/'

In [31]:
# collect all current txt files
txt_files = glob.glob(os.path.join(GCN_INPUT_FOLDER, "*.txt"))

In [32]:
d_list = []
exclude_files = []
for txt_data in txt_files: 
    
    d1 = read_txt_data(txt_data)
    d = {}
    
    try: 
        d2 = read_txt_data(d1['info_input_df'])
        d["txt_file"] = txt_data
        d.update(d1)
        d.update(d2)
        d_list.append(d)
    except Exception as e:
        exclude_files.append(txt_data)
        

df_overview = pd.DataFrame(d_list)   
df_overview.head()

,txt_file,max_num_veh,date,time,usr,env,path_input_df,info_input_df,sim_duration_timesteps,edge_creation_radius,...,conf_ID,conf_file,routes.flow.id,routes.flow.begin,routes.flow.end,routes.flow.probability,routes.flow.route,path_figure,path_df,routes.flow.number
0,/storage/remote/atcremers50/ss21_multiagentcon...,10,20210706-,17h23m10s,/usr/prakt/s0014,/usr/prakt/s0014/anaconda3/envs/carla_sumo_env,/storage/remote/atcremers50/ss21_multiagentcon...,/storage/remote/atcremers50/ss21_multiagentcon...,2000,3500,...,006,../004_data/cross_006/cross.rou.xml,flow1,0,2000,0.1,routedist2,../004_data/figures/plots_data/000006_veh_per_...,../004_data/dataframes/df_sim_000006.zip,NaN
1,/storage/remote/atcremers50/ss21_multiagentcon...,10,20210706-,17h23m10s,/usr/prakt/s0014,/usr/prakt/s0014/anaconda3/envs/carla_sumo_env,/storage/remote/atcremers50/ss21_multiagentcon...,/storage/remote/atcremers50/ss21_multiagentcon...,2000,3500,...,006,../004_data/cross_006/cross.rou.xml,flow1,0,2000,0.1,routedist2,../004_data/figures/plots_data/000006_veh_per_...,../004_data/dataframes/df_sim_000006.zip,NaN
2,/storage/remote/atcremers50/ss21_multiagentcon...,10,20210704-,12h04m16s,/usr/prakt/s0014,/usr/prakt/s0014/anaconda3/envs/carla_sumo_env,/storage/remote/atcremers50/ss21_multiagentcon...,/storage/remote/atcremers50/ss21_multiagentcon...,200,3500,...,003,../004_data/cross_003/cross.rou.xml,flow1,0,2000,NaN,routedist2,../004_data/figures/plots_data/000005_veh_per_...,../004_data/dataframes/df_sim_000005.zip,300
3,/storage/remote/atcremers50/ss21_multiagentcon...,10,20210706-,17h23m10s,/usr/prakt/s0014,/usr/prakt/s0014/anaconda3/envs/carla_sumo_env,/storage/remote/atcremers50/ss21_multiagentcon...,/storage/remote/atcremers50/ss21_multiagentcon...,2000,3500,...,006,../004_data/cross_006/cross.rou.xml,flow1,0,2000,0.1,routedist2,../004_data/figures/plots_data/000006_veh_per_...,../004_data/dataframes/df_sim_000006.zip,NaN
4,/storage/remote/atcremers50/ss21_multiagentcon...,10,20210704-,12h04m16s,/usr/prakt/s0014,/usr/prakt/s0014/anaconda3/envs/carla_sumo_env,/storage/remote/atcremers50/ss21_multiagentcon...,/storage/remote/atcremers50/ss21_multiagentcon...,200,3500,...,003,../004_data/cross_003/cross.rou.xml,flow1,0,2000,NaN,routedist2,../004_data/figures/plots_data/000005_veh_per_...,../004_data/dataframes/df_sim_000005.zip,300


In [33]:
df_overview.columns

Index(['txt_file', 'max_num_veh', 'date', 'time', 'usr', 'env',
       'path_input_df', 'info_input_df', 'sim_duration_timesteps',
       'edge_creation_radius', 'edge_maintenance_radius',
       'edge_weights_option', 'predict_after_timesteps', 'savestat',
       'delete_temporary_files', 'plotstat', 'path_GCN_input', 'path_GIF',
       'sim_duration_actualseconds', 'conf_ID', 'conf_file',
       '      routes.flow.id', '      routes.flow.begin',
       '      routes.flow.end', '      routes.flow.probability',
       '      routes.flow.route', 'path_figure', 'path_df',
       '      routes.flow.number'],
      dtype='object')

In [23]:
df_overview.predict_after_timesteps

0    2
1    2
Name: predict_after_timesteps, dtype: object

In [36]:
# store pkl file with GCN input 
file_name = f"OVERVIEW_TABLE"
file_to_write = open(f"../004_data/{file_name}.pkl", "wb")
pkl.dump(df_overview, file_to_write)


In [35]:
pd.read_pickle(f"../004_data/{file_name}.pkl")

,txt_file,max_num_veh,date,time,usr,env,path_input_df,info_input_df,sim_duration_timesteps,edge_creation_radius,...,conf_ID,conf_file,routes.flow.id,routes.flow.begin,routes.flow.end,routes.flow.probability,routes.flow.route,path_figure,path_df,routes.flow.number
0,/storage/remote/atcremers50/ss21_multiagentcon...,10,20210706-,17h23m10s,/usr/prakt/s0014,/usr/prakt/s0014/anaconda3/envs/carla_sumo_env,/storage/remote/atcremers50/ss21_multiagentcon...,/storage/remote/atcremers50/ss21_multiagentcon...,2000,3500,...,006,../004_data/cross_006/cross.rou.xml,flow1,0,2000,0.1,routedist2,../004_data/figures/plots_data/000006_veh_per_...,../004_data/dataframes/df_sim_000006.zip,NaN
1,/storage/remote/atcremers50/ss21_multiagentcon...,10,20210706-,17h23m10s,/usr/prakt/s0014,/usr/prakt/s0014/anaconda3/envs/carla_sumo_env,/storage/remote/atcremers50/ss21_multiagentcon...,/storage/remote/atcremers50/ss21_multiagentcon...,2000,3500,...,006,../004_data/cross_006/cross.rou.xml,flow1,0,2000,0.1,routedist2,../004_data/figures/plots_data/000006_veh_per_...,../004_data/dataframes/df_sim_000006.zip,NaN
2,/storage/remote/atcremers50/ss21_multiagentcon...,10,20210704-,12h04m16s,/usr/prakt/s0014,/usr/prakt/s0014/anaconda3/envs/carla_sumo_env,/storage/remote/atcremers50/ss21_multiagentcon...,/storage/remote/atcremers50/ss21_multiagentcon...,200,3500,...,003,../004_data/cross_003/cross.rou.xml,flow1,0,2000,NaN,routedist2,../004_data/figures/plots_data/000005_veh_per_...,../004_data/dataframes/df_sim_000005.zip,300
3,/storage/remote/atcremers50/ss21_multiagentcon...,10,20210706-,17h23m10s,/usr/prakt/s0014,/usr/prakt/s0014/anaconda3/envs/carla_sumo_env,/storage/remote/atcremers50/ss21_multiagentcon...,/storage/remote/atcremers50/ss21_multiagentcon...,2000,3500,...,006,../004_data/cross_006/cross.rou.xml,flow1,0,2000,0.1,routedist2,../004_data/figures/plots_data/000006_veh_per_...,../004_data/dataframes/df_sim_000006.zip,NaN
4,/storage/remote/atcremers50/ss21_multiagentcon...,10,20210704-,12h04m16s,/usr/prakt/s0014,/usr/prakt/s0014/anaconda3/envs/carla_sumo_env,/storage/remote/atcremers50/ss21_multiagentcon...,/storage/remote/atcremers50/ss21_multiagentcon...,200,3500,...,003,../004_data/cross_003/cross.rou.xml,flow1,0,2000,NaN,routedist2,../004_data/figures/plots_data/000005_veh_per_...,../004_data/dataframes/df_sim_000005.zip,300
